# `pygeodb` guide

In [1]:
%load_ext autoreload
%autoreload 2

import pygeodb

This notebook provides example utilization of the module with some example data reduced by Iolite 4. You will also need to have [iolite_tools](https://github.com/sarttiso/iolite_tools) to facilitate processing the example dataset into dataframes that are ready for `pygeodb`.

In [2]:
import iolite_tools

## Create `GeochemDB` Object

The core class for the `pygeodb` module is `GeochemDB`, which generates an object that allows you to interface with a geochemical SQLite database. The next well instatiates a `GeochemDB` object.

In [3]:
database_path = 'example_data/geochemdb_example.db'
geochemdb = pygeodb.GeochemDB(database_path)

## Adding Measurements

`GeochemDB.measurements_add()` adds new measurements to the `Measurements` table of the database.

**Note:** *`GeochemDB.measurements_add()` does not add samples to the `Samples` table.*

To add measurements to the database, you have to provide three dataframes:
1. **measurements** : This dataframe contains geochemical measurements as rows, alongside analysis names.
2. **analyses** : This dataframe lists analyses and metadata. All analyses referenced in measurements should be present here. No other analyses should be present.
3. **aliquots** : This datafram contains aliquots to which analyses belong. All aliquots referenced in the analsyes dataframe should be present here. No other aliquots should be present. 

Load in geochemical data from some source, and organize it as above. In this case, we're using `iolite_tools` to process reduced data exported into excel spreadsheets.  

In [11]:
df = iolite_tools.excel2measurements('example_data/2023-03_run-5_trace.xlsx',
                                     '2023-03', 5, 'trace')

df_measurements = iolite_tools.measurements2sql(df, refmat='91500')
df_analyses = iolite_tools.analyses2sql(df, date='2023-03-17',
                                        instrument='Nu Plasma 3D',
                                        technique='LASS ICPMS')
df_aliquots = iolite_tools.aliquots2sql(df, material='zircon')

In [12]:
geochemdb.measurements_add(df_measurements, df_analyses, df_aliquots)

Sample names not matched:
{'AT22 FRAN 17 29', 'NIST612', 'Ples', 'GJ1', '91500'}
Added:
122 aliquots,
122 analyses,
3416 measurements


## Updating Measurements

`GeochemDB.measurements_update()` updates existing measurements in the `Measurements` table of the database. It does not add any rows to any table, and it only requires a measurements dataframe like the one created above.

In [6]:
geochemdb.measurements_update(df_measurements)

Updated:
3416 measurements


## Get Measurements by Sample

In [13]:
df_measurements = geochemdb.measurements_by_sample('AT2310 16.4')